In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/My Drive')
!pwd

/content/drive/My Drive


In [ ]:
%tensorflow_version

Currently selected TF version: 2.x
Available versions:
* 1.x
* 2.x


In [ ]:
import pandas as pd
import numpy as np
from skimage.io import imread
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
# %matplotlib inline

import keras
from keras import optimizers
from keras.utils import to_categorical
from keras.layers import Input, Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Activation, concatenate
from keras.layers.advanced_activations import ReLU
from keras.models import Model
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint

In [ ]:
def load_dataset():
  train_data = pd.read_csv('trainset.csv')
  y_train = pd.read_csv('data1.csv')
  target_data = pd.read_csv('target_data.csv')
  # target_label = pd.read_csv('target_label.csv')
  return train_data, y_train, target_data

In [ ]:
# train_data.head()
# y_train.head()

In [ ]:
train_data, y_train, target_data = load_dataset()

In [ ]:
# train_data.shape
# target_data.shape

In [ ]:
num_classes = 7
train_data = np.array(train_data.iloc[:,:])
train_data = np.array([np.reshape(i,(128,128)) for i in train_data])
target_data = np.array(target_data.iloc[:,:])
target_data = np.array([np.reshape(i,(128,128)) for i in target_data])

In [ ]:
train_data.shape
target_data.shape

(333, 128, 128)

In [ ]:
train_data = train_data.astype('float32')
train_data = train_data/255

rgb_batch = np.repeat(train_data[..., np.newaxis], 3, -1)
rgb_batch.shape

target_data = target_data.astype('float32')
target_data = target_data/255

rgb_target = np.repeat(target_data[..., np.newaxis], 3, -1)
rgb_target.shape

(333, 128, 128, 3)

In [ ]:
target_data = rgb_target

In [ ]:
target_data.shape

(333, 128, 128, 3)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(rgb_batch, y_train, test_size = 0.3, random_state=666)
target_xtrain, target_xtest = train_test_split(target_data, test_size = 0.3, random_state=666)

In [ ]:
input_shape = (128, 128, 3)

In [ ]:
def VGG_16(input_image):
    model = Conv2D(64, (3,3), padding="same", input_shape = input_shape)(input_image)
    model = ReLU(max_value=1.0)(model)
    # model = Conv2D(64, (3,3), padding="same")(model)
    # model = ReLU(max_value=1.0)(model)
    model = MaxPooling2D(pool_size=(2,2), padding="same")(model)
    # model = Dropout(0.25)(model)

    model = Conv2D(128, (3,3), padding="same")(model)
    model = ReLU(max_value=1.0)(model)
    model = Conv2D(128, (3,3), padding="same")(model)
    model = ReLU(max_value=1.0)(model)
    model = MaxPooling2D(pool_size=(2,2), padding="same")(model)
    model = Dropout(0.25)(model)

    # model = Conv2D(256, (3,3), padding="same")(model)
    # model = ReLU(max_value=1.0)(model)
    model = Conv2D(256, (3,3), padding="same")(model)
    model = ReLU(max_value=1.0)(model)
    model = MaxPooling2D(pool_size=(2,2), padding="same")(model)
    model = Dropout(0.25)(model)


    return model

In [ ]:
current_input = Input(shape=input_shape)
current_model = VGG_16(current_input)

target_input = Input(shape=input_shape)
target_model = VGG_16(target_input)

In [ ]:
conv = concatenate([current_model, target_model])

In [ ]:
conv = Flatten()(conv)
dense = Dense(512)(conv)
dense = ReLU(max_value=1.0)(dense)
dense = Dropout(0.25)(dense)
# dense = Dense(512)(dense)
# dense = ReLU(max_value=1.0)(dense)
# dense = Dropout(0.5)(dense)

output = Dense(num_classes, activation='tanh')(dense)

model = Model(inputs=[current_input, target_input], outputs=[output])

opt = SGD(learning_rate=0.01)

model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])


In [ ]:
best_weights_file="weights.best.hdf5"
checkpoint = ModelCheckpoint(best_weights_file, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks = [checkpoint]

In [ ]:
model.fit([x_train, target_xtrain], y_train,
          epochs=15,
          validation_data=([x_test, target_xtest], y_test),
          callbacks=callbacks,
          verbose=1)

Epoch 1/15
8/8 [==============================] - 34s 278ms/step - loss: 15.7407 - accuracy: 0.5451 - val_loss: 14.6341 - val_accuracy: 0.6700

Epoch 00001: val_accuracy improved from -inf to 0.67000, saving model to weights.best.hdf5
Epoch 2/15
8/8 [==============================] - 1s 88ms/step - loss: 15.5502 - accuracy: 0.7637 - val_loss: 14.6409 - val_accuracy: 0.6700

Epoch 00002: val_accuracy did not improve from 0.67000
Epoch 3/15
8/8 [==============================] - 1s 86ms/step - loss: 15.6285 - accuracy: 0.7597 - val_loss: 14.7400 - val_accuracy: 0.6700

Epoch 00003: val_accuracy did not improve from 0.67000
Epoch 4/15
8/8 [==============================] - 1s 86ms/step - loss: 15.9161 - accuracy: 0.7685 - val_loss: 14.6840 - val_accuracy: 0.6700

Epoch 00004: val_accuracy did not improve from 0.67000
Epoch 5/15
8/8 [==============================] - 1s 88ms/step - loss: 15.6059 - accuracy: 0.7276 - val_loss: 14.6867 - val_accuracy: 0.6700

Epoch 00005: val_accuracy did no

In [ ]:
model.evaluate([x_test, target_xtest], y_test)

4/4 [==============================] - 0s 20ms/step - loss: 14.4594 - accuracy: 0.6700


[14.459378242492676, 0.6700000166893005]

In [ ]:
prediction = model.predict([x_test, target_xtest])
# prediction = np.argmax(prediction[10])

In [ ]:
print(prediction)

[[ 9.84908283e-01 -4.08849008e-02 -9.89484072e-01  4.66970265e-01
   1.05800189e-01  1.67856604e-01  9.16396379e-02]
 [ 9.92205203e-01 -3.88330609e-01 -9.94253993e-01  5.42590857e-01
   9.44996998e-02  8.35585743e-02  7.60527775e-02]
 [ 9.68952954e-01  3.84597719e-01 -9.78712320e-01  3.84883404e-01
   1.21202044e-01  2.32258514e-01  8.74703676e-02]
 [ 9.77179229e-01  1.54018015e-01 -9.84513581e-01  4.28686947e-01
   9.89760906e-02  1.86423048e-01  8.03595334e-02]
 [ 9.90933299e-01 -2.96536952e-01 -9.93307531e-01  5.22805274e-01
   9.88750011e-02  9.49409306e-02  8.57084095e-02]
 [ 9.81587172e-01  1.08045988e-01 -9.87598240e-01  4.40004706e-01
   1.23697571e-01  1.64365754e-01  9.32225659e-02]
 [ 9.92372394e-01 -4.55821633e-01 -9.94494438e-01  5.53294897e-01
   7.91994408e-02  7.71674886e-02  6.76269680e-02]
 [ 9.69987392e-01  3.31764460e-01 -9.79657292e-01  3.81705582e-01
   1.05609372e-01  2.08229974e-01  8.65168571e-02]
 [ 9.93171275e-01 -4.87989038e-01 -9.94942665e-01  5.60449600e-0

In [ ]:
# import cv2
# from google.colab.patches import cv2_imshow
# img1 = cv2.imread('data65_27.png')
# img2 = cv2.imread('target_image_1.png')
#mg1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
# img1 = cv2.resize(img1, dsize=(128,128), interpolation = cv2.INTER_CUBIC)
# plt.imshow(img1)
# img1.shape

# img1 = np.expand_dims(img1, axis = 0)
# img1 = np.expand_dims(img1, axis = 3)
# img1 = img1.astype('float32')
# img1 = img1/255
# plt.imshow(img1.reshape(128,128), cmap = plt.cm.binary)

In [ ]:
# from PIL import Image
# # Open the image form working directory
# img1 = Image.open('/content/data65_27.png')
# img2 = Image.open('/content/target_image_1.png')

In [ ]:
# from matplotlib import image
# from matplotlib import pyplot
# # load image as pixel array
# img1 = image.imread('/content/data65_27.png')
# img2 = image.imread('/content/target_image_1.png')

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
img = cv2.imread('/content/data65_27.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img= cv2.resize(img, dsize=(128,128), interpolation = cv2.INTER_CUBIC)
# plt.imshow(img)
img.shape

img = np.reshape(img, (1, 128, 128))
# img = np.expand_dims(img, axis = 0)
# img = np.expand_dims(img, axis = 3)
# img = np.reshape(128,128)
# img = img.astype('float32')
img = img/255
img = np.repeat(img[..., np.newaxis], 3, -1)
# plt.imshow(img.reshape(128,128), cmap = plt.cm.binary)

error: ignored

In [ ]:
img.shape

In [ ]:
img2 = cv2.imread('/content/target_image_1.png')
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
img2= cv2.resize(img2, dsize=(128,128), interpolation = cv2.INTER_CUBIC)
# plt.imshow(img)
img2.shape

img2 = np.reshape(img2, (1, 128, 128))
# img = np.expand_dims(img, axis = 0)
# img = np.expand_dims(img, axis = 3)
# img = np.reshape(128,128)
# img = img.astype('float32')
img2 = img2/255
img2 = np.repeat(img2[..., np.newaxis], 3, -1)
# plt.imshow(img.reshape(128,128), cmap = plt.cm.binary)

In [ ]:
img2.shape

In [ ]:
predictionimg = model.predict([img,img2])

In [ ]:
predictionimg